<a href="https://colab.research.google.com/github/ValentinoVizner/Python_Deep_Dive_1/blob/master/Section_3_Variables_and_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SECTION 3. VARIABLES AND MEMORY

## 1. Variables Are Memory References

![alt text](https://drive.google.com/uc?id=1VdPKXbWvyWOzxXhgh12LnQWaySAmVhCn)

When we store data in memory addresses we may actually use more than 1 slot at a time (object 1 and object 2).
</br>
But as long as we know where the object starts in memory thats good enough.
</br>
So the object 1 starts and 0x1000 and it overflows into another memory address e.g. 0x1001
</br>
Object 2 starts at memory address 0x1002 and it overflows into 2 more memory address.
</br>
While Object 3 fits preciselly into one slot of memory.
</br>
</br>
**Heap** is range of memory addresses where our objects are stored.
**Python Memory Manager** is responsible of pulling object from the **heap**

Here is an example:

![alt text](https://drive.google.com/uc?id=18UOEEgkBpnZqZDmr-Fha7t70wUukcxcd)

We can find the memory address that a variable *references*, by using the `id()` function.

The `id()` function returns the memory address of its argument as a base-10 integer.

We can use the function `hex()` to convert the base-10 number to base-16.

In [0]:
my_var = 10
print('my_var = {0}'.format(my_var))
print('memory address of my_var (decimal): {0}'.format(id(my_var)))
print('memory address of my_var (hex): {0}'.format(hex(id(my_var))))

In [0]:
greeting = 'Hello'
print('greeting = {0}'.format(greeting))
print('memory address of my_var (decimal): {0}'.format(id(greeting)))
print('memory address of my_var (hex): {0}'.format(hex(id(greeting))))

Note how the memory address of `my_var` is **different** from that of `greeting`.

Strictly speaking, `my_var` is not "equal" to 10. 

Instead `my_var` is a **reference** to an (*integer*) object (*containing the value 10*) located at the memory address `id(my_var)`

Similarly for the variable `greeting`.

## 2. Reference Counting

![alt text](https://drive.google.com/uc?id=1tC5p8CU48NWqyRoeVlxBky7JbUudg2yo)

REMEMBER: We are dealing with pointers, so our other_var is pointing, i.e. REFERENCING to the memory address 0x1000 and its not actually REFERENCING to the value 10.
</br>
They are both pointing to the same reference, so the reference count is 2.

Now if we delete the `my_var` the `count` will decrease to `1`.
</br>
After we delete last variable referencing to the memory address i.e. `other_var` our `count` will be `0` and **Python Memory Manager** will delete and free up memory space/address for someone else, another variable to use it.

![alt text](https://drive.google.com/uc?id=1Zr4LR4XYFKQ9ilOxFchLCMtQeFGGT8xQ)
</br>
</br>
There is the downside for using `sys.getrefcount(my_var)` because when we are passing `my_var` to `sys.getrefcount(my_var)` it creates and extra reference i.e. count

![alt text](https://drive.google.com/uc?id=1ZaVrmuDDRfrPL1UTM0hBZGAxQ5F6y5Ci)
</br>
</br>

This is much better because it is not referencing the variable, instead we are just passing the memory address, so it does NOT affect REFERENCING COUNT!

In [0]:
import sys

a = [1, 2, 3]

sys.getrefcount(a)

2

But why is this returning 2, instead of the expected 1 we obtained with the previous function?

Answer: The `sys.getrefcount()` function takes `my_var` as an argument, this means it receives (and stores) a reference to `my_var`'s memory address **also** - hence the count is off by 1.

In [0]:
# BETTER WAY
import ctypes

def ref_count(address: int):
    return ctypes.c_long.from_address(address).value

my_var = [1, 2, 3, 4]

ref_count(id(my_var))

1

In [0]:
other_var = my_var

print(id(my_var))
print(id(other_var))

140195183375240
140195183375240


In [0]:
ref_count(id(other_var))

2

In [0]:
other_other = my_var

ref_count(id(other_var))

3

In [0]:
other_other = 100

In [0]:
ref_count(id(other_other))

189

You'll probably never need to do anything like this in Python. Memory management is completely transparent - this is just to illustrate some of what is going behind the scenes as it helps to understand upcoming concepts.

In [0]:
my_var_id = id(my_var)
other_var = None
my_var = None


ref_count(my_var_id)

64142

In [0]:
ref_count(my_var_id)

64140

What's happening here is that once the memory address is freed, we do not know what is stored there, probably JUNK. It's dangerous to manage/delete that in Python

## 3. Garbage Collection

![alt text](https://drive.google.com/uc?id=1ig7qPO-YVKlP13KBbKbJuF7sqNupNOIk)
</br>
What happens if we remove the `my_var` pointer?
</br>
Then the reference count of object A goes to 0, but the reference count of object B is still 1.
</br>
BUT the reference count of object A is going to 0 and it will be destroyed, once it gets destroyed, reference count of object B will go to 0 and it gets destroyed as well.
</br>
So far that is good, what we expected to happen.
</br>
</br>
Consider now that object B has `var_2` that points to `var_1`, i.e. Circular References:


![alt text](https://drive.google.com/uc?id=11SbqFbGcF7fyqe0zku2Ivc5ndpbFFsyt)
</br>
Now when we remove the `my_var` the reference count for `var_1` in object A will still be 1.
</br>
So in this case by removing `my_var` both variables in objects A and B wont be destroyed as expected.

![alt text](https://drive.google.com/uc?id=1naSNs4OJ5Xlc6aOZXdoR__DXRDBxvCap)
</br>
Beccause **Python Memory Manager** can not clean this up and we leave the thins as is, we will have **MEMORY LEAK!!**
</br>
</br>
Thats where the **GARBAGE COLLECTOR** comes in!

![alt text](https://drive.google.com/uc?id=1hhcGms4SQf83N5LDCXbmSQL4x78K9IR5)

![alt text](https://drive.google.com/uc?id=1DbKKhD0-bK-jdbKhF8cR3z6jj4lMA3fJ)

In [0]:
import ctypes
import gc

In [0]:
def ref_count(address: int):
    return ctypes.c_long.from_address(address).value

We create a function that will search the objects in the GC for a specified id and tell us if the object was found or not:

In [0]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not Found"

Next we define two classes that we will use to create a circular reference

Class A's constructor will create an instance of class B and pass itself to class B's constructor that will then store that reference in some instance variable.

In [0]:
class A:
    def __init__(self):
        self.b = B(self)
        print(f"A: self: {hex(id(self))}, b: {hex(id(self.b))}")

In [0]:
class B:
    def __init__(self, a):
        self.a = a
        print(f"B: self: {hex(id(self))}, a: {hex(id(self.a))}")

We turn off the GC so we can see how reference counts are affected when the GC does not run and when it does (by running it manually).

In [0]:
gc.disable()

In [0]:
my_var = A()

B: self: 0x7f81bc103128, a: 0x7f81bc1030f0
A: self: 0x7f81bc1030f0, b: 0x7f81bc103128


In [0]:
hex(id(my_var))

'0x7f81bc1030f0'

In [0]:
print('a: \t{0}'.format(hex(id(my_var))))
print('a.b: \t{0}'.format(hex(id(my_var.b))))
print('b.a: \t{0}'.format(hex(id(my_var.b.a))))

a: 	0x7f81bc1030f0
a.b: 	0x7f81bc103128
b.a: 	0x7f81bc1030f0


In [0]:
a_id = id(my_var)
b_id = id(my_var.b)

In [0]:
ref_count(a_id)

2

In [0]:
ref_count(b_id)

1

In [0]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 2
refcount(b) = 1
a: Object exists
b: Object exists


As we can see the A instance has two references (one from `my_var`, the other from the instance variable `b` in the B instance)

The B instance has one reference (from the A instance variable `a`)

Now, let's remove the reference to the A instance that is being held by `my_var`:

In [0]:
my_var= None

In [0]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 1
refcount(b) = 1
a: Object exists
b: Object exists


As we can see, the reference counts are now both equal to 1 (a pure circular reference), and reference counting alone did not destroy the A and B instances - they're still around. If no garbage collection is performed this would result in a memory leak.

</br>
</br>
Let's run the GC manually and re-check whether the objects still exist:

In [0]:
gc.collect()
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 3112880742897218958
refcount(b) = 0
a: Not Found
b: Not Found


In [0]:
ref_count(a_id)

3112880742897218958

In [0]:
ref_count(b_id)

0

In [0]:
object_by_id(b_id)

'Not Found'

So why do we get some number if we garbage collected b_id and a_id.
</br>
Variables got destroyed, their memory addresses, so we do not know what are those values, certainly they are not memory addresses of variables we saw earlier.

## 4. Dynamic vs Static Typing

![alt text](https://drive.google.com/uc?id=1D8zc-W3_MG3zIdW-6hWxsmifgwYeBa9O)

In [0]:
a = 'hello'

In [0]:
type(a)

str

In [0]:
a = lambda x: x**2

In [0]:
a(2)

4

In [0]:
type(a)

function

In [0]:
a = 3 + 4j

In [0]:
type(a)

complex

## 5.Variable Re-Assignment

![alt text](https://drive.google.com/uc?id=1-oB-43dG9yjAt73PAAACw-ANxCFf06Xa)
</br>
This is very important to remember, that we are not Re-Assigning the variable, we are creating whole new memory address, so 2 different values, i.e. memory addresses are used for the same variable `my_var`
</br>
So what happens if we do like `my_var = my_var + 5`, will it create new address in memory, i.e. reserve memory:

![alt text](https://drive.google.com/uc?id=1CeJSPNy9ZTqfoJXZaMw5R5K5XfI6AE0i)
</br>
Yes it will reserve new memory address.

In [0]:
a = 10

In [0]:
hex(id(a))

'0xa68be0'

In [0]:
a = 15
hex(id(a))

'0xa68c80'

In [0]:
a = 5
hex(id(a))

'0xa68b40'

In [0]:
a = a + 1
hex(id(a))

'0xa68b60'

However, look at this:

In [0]:
a = 10
b = 10
print(hex(id(a)))
print(hex(id(b)))

0xa68be0
0xa68be0


The memory addresses of both **a** and **b** are the same!! 

We'll revisit this in a bit to explain what is going on.

## 6.Object Mutability

![alt text](https://drive.google.com/uc?id=1yqXk4jiO4ZNRBPpC3OY0vrDA6EYZ3SSL)

![alt text](https://drive.google.com/uc?id=1nrJeE1mTyJ5wzOM8jCyg8eAuc0MKUnRx)
</br>
User-defined cclasses can be both Immutable and Mutable, depends how we define internal methods, etc.

![alt text](https://drive.google.com/uc?id=1VV-__LmvOKikCk9HzQqGbdSRCGRnEVNN)
</br>
</br>
Here we have to be careful, because although **tuple** is **immutable** if we put **list** inside **tuple** then we can mutate alements of the list.
</br>
So in some way tuple is mutable and not completly frozen as it should be.

![alt text](https://drive.google.com/uc?id=1qaM91o6wM2v9icO8Sz1dOA7VprJeAbnD)

In [0]:
my_list = [1, 2, 3]
print(my_list)
print(hex(id(my_list)))

[1, 2, 3]
0x7f7aa5993488


In [0]:
my_list.append(4)
print(my_list)
print(hex(id(my_list)))

[1, 2, 3, 4]
0x7f7aa5993488


In [0]:
my_list_1 = [1, 2, 3]
print(my_list_1)
print(hex(id(my_list_1)))

[1, 2, 3]
0x7f7aa59217c8


In [0]:
my_list_1 = my_list_1 + [4]
print(my_list_1)
print(hex(id(my_list_1)))

[1, 2, 3, 4]
0x7f7aa58b7408


Notice here that the memory address of *my_list_1* **did** change.

This is because concatenating two lists objects *my_list_1* and *[4]* did not modify the contents of *my_list_1* - instead it created a new list object and re-assigned *my_list_1* to reference this new object.

Similarly with **dictionary** objects that are also **mutable** types.

In [0]:
my_dict = dict(key1='value 1', key2='value 2')
print(my_dict)
print(hex(id(my_dict)))

{'key1': 'value 1', 'key2': 'value 2'}
0x7f7aa58d23a8


In [0]:
my_dict['key1'] = 'modified value 1'
print(my_dict)
print(hex(id(my_dict)))

{'key1': 'modified value 1', 'key2': 'value 2'}
0x7f7aa58d23a8



Now consider the immutable sequence type: **tuple**
</br>
</br>
</br>
The tuple is immutable, so elements cannot be added, removed or replaced.

In [0]:
t = (1, 2, 3)

This tuple will **never** change at all. It has three elements, the integers 1, 2, and 3. This will remain the case as long as **t**'s reference is not changed.
</br>
Everything in Python is object, so even numbers are, here is the proof:

In [0]:
id(t[0])

10914496

In [0]:
id(t[1])

10914528

In [0]:
a = [1, 2]
b = [3, 4]
t = (a, b)

Now, **t** is still immutable, i.e. it contains a reference to the object **a** and the object **b**. **That** will never change as long as **t**'s reference is not re-assigned.

**However**, the elements **a** and **b** are, themselves, mutable.

In [0]:
a.append(3)
b.append(5)
print(t)

([1, 2, 3], [3, 4, 5])


Observe that the contents of **a** and **b** **did** change!

So immutability can be a little more subtle than just thinking something can never change. 

The tuple **t** did **not** change - it contains two elements, that are the references **a** and **b**. And that will not change. But, because the referenced elements are mutable themselves, it appears as though the tuple has changed.

It hasn't though - that distinction is subtle but important to understand!

## 7.Function Arguments and Mutability

![alt text](https://drive.google.com/uc?id=1q0_nii_9vLQ4O4JCJ8fvkjuqHdea8ddq)

![alt text](https://drive.google.com/uc?id=1mhdXDW1-MmleC-y186LkZkuC7VwdWGWQ)
</br>
</br>
Nakon što pustimo funkciju `process()`, ono što je dobro je to što kada napravimo `print(my_var)` onda ne dobijemo `hello world`, dakle ne mutira varijabla `my_var`.
</br>
Također, `s` ne odlazi u staru memorijsku adresu, već u novu memorijsku adresu,
</br>
</br>
Kao što vidimo, kako je `string` **immutable** imamo sigurnost te tako funkcija u koju dajemo string, taj string neće biti promijenjen od strane funkcije.
</br>
ALI moramo pazit na tipa onu situaciju kao kod **tupla** jer je to **kontejnerski** tip u kojem pohranjujemo podatke.


![alt text](https://drive.google.com/uc?id=1DiTYbuzV1vtFqbxY8CiltM1wCfXtOJ-K)
</br>
</br>

Ovdej vidimo kako funkcija `process()` mijenja **MUTIRA** listu te kada printamo `my_list` prije i poslije pokretanje funkcije dobijemo dvije različite liste.
</br>
Međutim, to je ta ista lista, na istoj memorijskoj adresi, ali je sada drugačija, odnosno imamo **side effect**, nešto što se nastoji izbjeći kod primjerice **FUNKCIONALNOG PROGRAMIRANJA**.
</br>
Možemo dobiti ne željene promjene ako nismo svjesni ove promjene, tj. činjenice kako je `list` **MUTABLE**.

![alt text](https://drive.google.com/uc?id=1zhzPVy8jeFMoOrYkdnnKJUTjQUqnE9t6)

In [0]:
def proccess(s):
    print(f"Initial s # = {hex(id(s))}")
    s = s + " world"
    print(f"Final s # = {hex(id(s))}")

In [0]:
my_var = "hello"
print('my_var # = {0}'.format(hex(id(my_var))))

my_var # = 0x7f7aa50283e8



After we "modify" *s*, *s* is pointing to a new memory address:

In [0]:
proccess(my_var)

Initial s # = 0x7f7aa50283e8
Final s # = 0x7f7aa4f2b070


And our own variable *my_var* is still pointing to the original memory address:

In [0]:
print('my_var # = {0}'.format(hex(id(my_var))))

my_var # = 0x7f7aa50283e8


Let's see how this works with mutable objects:

In [0]:
def modify_list(items):
    print('initial items # = {0}'.format(hex(id(items))))
    if len(items) > 0:
        items[0] = items[0] ** 2
    items.pop() # removes the last element in the list
    items.append(5)
    print('final items # = {0}'.format(hex(id(items))))

In [0]:
my_list = [2, 3, 4]
print('my_list # = {0}'.format(hex(id(my_list))))

my_list # = 0x7f7aa4f32348


In [0]:
modify_list(my_list)

initial items # = 0x7f7aa4f32348
final items # = 0x7f7aa4f32348


In [0]:
hex(id(my_list))

'0x7f7aa4f32348'

In [0]:
my_list

[4, 3, 5]

As you can see, throughout all the code, the memory address referenced by *my_list* and *items* is always the **same** (shared) reference - we are simply modifying the contents (**internal state**) of the object at that memory address.

Now, even with immutable container objects we have to be careful, e.g. a tuple containing a list (the tuple is immutable, but the list element inside the tuple **is** mutable)

In [0]:
def modify_tuple(t):
    print('initial t # = {0}'.format(hex(id(t))))
    t[0].append(100)
    print('final t # = {0}'.format(hex(id(t))))

In [0]:
my_tuple = ([1, 2], 'a')

In [0]:
hex(id(my_tuple))

'0x7f7aa598bf08'

In [0]:
modify_tuple(my_tuple)

initial t # = 0x7f7aa598bf08
final t # = 0x7f7aa598bf08


In [0]:
my_tuple

([1, 2, 100], 'a')

As you can see, the first element of the tuple was mutated.

## 8.Shared References and Mutability

![alt text](https://drive.google.com/uc?id=1VIKOcFlMwA8Ri6cerQbXvJVUjZTvnhCQ)

![alt text](https://drive.google.com/uc?id=1hD5NiJZQZ0F4rUI9F8OuIXxY35DamLeb)
</br>
</br>
The thing Python does for optimization shown in this picture is actually safe cause the objects stored in memory address are **IMMUTABLE**.

![alt text](https://drive.google.com/uc?id=1n4j7VTsSZZaTVe436U03YbaI_Qzua6sm)
</br>
</br>
As we see if **mutable** objects have the same values assigned to variable, they will be stored in different memory address.
</br>
Which is logical because we can **mutate** them so they won't be the same all the time.

In [0]:
my_var_1 = 'hello'
my_var_2 = my_var_1
print(my_var_1)
print(my_var_2)

hello
hello


In [0]:
print(hex(id(my_var_1)))
print(hex(id(my_var_2)))

0x7f7aa50283e8
0x7f7aa50283e8


In [0]:
my_var_2 = my_var_2 + ' world!'

In [0]:
print(hex(id(my_var_1)))
print(hex(id(my_var_2)))

0x7f7aa50283e8
0x7f7aa4f32630


Be careful if the variable type is mutable!

Here we create a list (*my_list_1*) and create a variable (*my_list_2*) referencing the same list object:

In [0]:
my_list_1 = [1, 2, 3]
my_list_2 = my_list_1
print(my_list_1)
print(my_list_2)

[1, 2, 3]
[1, 2, 3]


As we can see they have the same memory address (shared reference):

In [0]:
print(hex(id(my_list_1)))
print(hex(id(my_list_2)))

0x7f7aa4f3e7c8
0x7f7aa4f3e7c8


Now we modify the list referenced by `my_list_2`:

In [0]:
my_list_2.append(4)

In [0]:
print(my_list_2)

[1, 2, 3, 4]


And since my_list_1 references the same list object, it has also changed:

As you can see, both variables still share the same reference:

In [0]:
print(hex(id(my_list_1)))
print(hex(id(my_list_2)))

0x7f7aa4f3e7c8
0x7f7aa4f3e7c8


## 9.Behind the scenes with Python's memory manager
----

In [0]:
a = 10
b = 10

In [0]:
print(hex(id(a)))
print(hex(id(b)))

0xa68be0
0xa68be0


Same memory address!!

This is safe for Python to do because integer objects are **immutable**. 

So, even though *a* and *b* initially shared the same mempry address, we can never modify *a*'s value by "modifying" *b*'s value. 

The only way to change *b*'s value is to change it's reference, which will never affect *a*.

In [0]:
b = 15

In [0]:
print(hex(id(a)))
print(hex(id(b)))

0xa68be0
0xa68c80


However, for mutable objects, Python's memory manager does not do this, since that would **not** be safe.

In [0]:
my_list_1 = [1, 2, 3]
my_list_2 = [1, 2 , 3]

As you can see, although the two variables were assigned identical "contents", the memory addresses are not the same:

In [0]:
print(hex(id(my_list_1)))
print(hex(id(my_list_2)))    

0x7f7aa4f3e608
0x7f7aa4f3e3c8


## 10.Variable Equality

![alt text](https://drive.google.com/uc?id=1lrGiju8DkFnSv--b1pjsskcg20bmEh-o)
![alt text](https://drive.google.com/uc?id=1NG0u_auxFCtBiixA9tiNT763D3j1InRI)

![alt text](https://drive.google.com/uc?id=1L2c2P7WPDx3HVEd5Hy_jbgZKJlWRHUzd)
</br>
Now remember `None` is NOT **nothing**, it is an actual object ii Python.

In [0]:
a = 10
b = 10

print(hex(id(a)))
print(hex(id(b)))

0xa68be0
0xa68be0


When we use the **is** operator, we are comparing the memory address **references**:

In [0]:
print("a is b: ", a is b)

a is b:  True


The following however, do not have a shared reference:

In [0]:
a = [1, 2, 3]
b = [1, 2, 3]

print(hex(id(a)))
print(hex(id(b)))

0x7fca762960c8
0x7fca762934c8


Although they are not the same objects, they do contain the same "values":

In [0]:
print("a is b: ", a is b)
print("a == b", a == b)

a is b:  False
a == b True


Python will attempt to compare values as best as possible, for example:

In [0]:
a = 10
b = 10.0

These are **not** the same reference, since one object is an **int** and the other is a **float**

In [0]:
print(type(a))
print(type(b))
print(hex(id(a)))
print(hex(id(b)))
print('a is b:', a is b)
print('a == b:', a == b)

<class 'int'>
<class 'float'>
0xa68be0
0x7fca76233648
a is b: False
a == b: True


So, even though *a* is an integer 10, and *b* is a float 10.0, the values will still compare as equal.
</br>
</br>
In fact, this will also have the same behavior:

In [0]:
c = 10 + 0j
print(type(c))
print('a is c:', a is c)
print('a == c:', a == c)

<class 'complex'>
a is c: False
a == c: True


### The None Object
----

**None** is a built-in "variable" of type *NoneType*.

Basically the keyword **None** is a reference to an object instance of *NoneType*.

NoneType objects are immutable! Python's memory manager will therefore use shared references to the None object.

In [0]:
print(None)
hex(id(None))
type(None)


a = None
print(type(a))
print(hex(id(a)))

None
<class 'NoneType'>
0x9d4380


In [0]:
a is None

True

In [0]:
a == None

True

In [0]:
b = None
hex(id(b))

'0x9d4380'

In [0]:
a is b

True

In [0]:
a == b

True

In [0]:
l = []
type(l)

list

In [0]:
l is None

False

In [0]:
l == None

False

## 11.Everything is and Object

![alt text](https://drive.google.com/uc?id=1p_h7Kee_o2B-9vNezsILTyVoF7JK2kZU)
![alt text](https://drive.google.com/uc?id=1a0NK0KfEqjLSIBKGnFK8y1EDtPmkOSTj)
</br>
</br>
</br>

What actually make Python powerful are next consequences of everythin as an object:

![alt text](https://drive.google.com/uc?id=1cFG1R2Rd_b3HFfZ1NX_kmaIEPvGT01-m)
</br>
</br>
</br>
There is also one thing to note:

![alt text](https://drive.google.com/uc?id=18vzVaAophDNK1eg4aSCHjlDExrMEE7mf)

We can even request the class documentation:

In [1]:
help(int)

Help on class int in module builtins:

class int(object)
 |  int(x=0) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil__(...)
 |      Ceiling of

As we see from the docs, we can even create an **int** using an overloaded constructor:

In [2]:
b = int('10', base=2)
print(b)
print(type(b))

2
<class 'int'>


### Functions are Objects too
---

In [4]:
def square(a):
    return a ** 2

type(square)

function

In fact, we can even assign them to a variable:

In [8]:
f = square
type(f)
print(f"f is square \n{f is square}")

f is square 
True


A function can return a function

In [7]:
def cube(a):
    return a ** 3

def select_function(fn_id):
    if fn_id == 1:
        return square
    else:
        return cube

f = select_function(1)
print(hex(id(f)))
print(hex(id(square)))
print(hex(id(cube)))
print(type(f))
print('f is square: ', f is square)
print('f is cube: ', f is cube)
print(f)
print(f(2))

0x7ff538b55f28
0x7ff538b55f28
0x7ff538b51730
<class 'function'>
f is square:  True
f is cube:  False
<function square at 0x7ff538b55f28>
4


In [9]:
f = select_function(2)
print(hex(id(f)))
print(hex(id(square)))
print(hex(id(cube)))
print(type(f))
print('f is square: ', f is square)
print('f is cube: ', f is cube)
print(f)
print(f(2))

0x7ff538b51730
0x7ff538b55f28
0x7ff538b51730
<class 'function'>
f is square:  False
f is cube:  True
<function cube at 0x7ff538b51730>
8


We could even call it this way:

In [10]:
select_function(1)(5)

25

A Function can be passed as an argument to another function

(This example is pretty useless, but it illustrates the point effectively)

In [11]:
def exec_function(fn, n):
    return fn(n)

result = exec_function(cube, 2)
print(result)

8


In [13]:
select_function(2)(5) # Here we determine to use CUBE function and pass it value of 5, i.e. 5**3

125

## 12.Python OPtimization: INTERNING

![alt text](https://drive.google.com/uc?id=1nnJ8TsRgZXVMixVHvydCuXhav2EGaPuk)
</br>
In this Python Course 3.6. i used.

![alt text](https://drive.google.com/uc?id=1Q07_owH20mNdcVeYuO8R3jdXJ8LdlMX_)
</br>
</br>
</br>
But wait WTF!?!?!?